#### Problema:

Se requiere construir un sistema de recomendación de libros basados en los resumenes de libros y los temas (tópicos) de los mismos.

Para tal fin, se utilizará eñ [CMU Book Summary Dataset](https://www.cs.cmu.edu/~dbamman/booksummaries.html)

## Librerias

In [6]:
import csv 
import json
import pickle

import pandas as pd
import numpy as np
from collections import Counter # Para contar frecuencias

# Preprocesar texto
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Modelado de tópicos 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Visualizaciones
import pyLDAvis
import matplotlib.pyplot as plt
import seaborn as sns
import pyLDAvis.lda_model

import warnings
warnings.filterwarnings('ignore', category = DeprecationWarning) # Al instalar pyLDAvis ocasiona un warning con ipkernel

import sklearn
for lib in [sklearn, pyLDAvis, np, pd]:
    print(lib.__name__, lib.__version__)

sklearn 1.2.1
pyLDAvis 3.4.0
numpy 1.26.4
pandas 2.2.3


In [7]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/danielml/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
!tar -xzvf "/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/booksummaries.tar.gz" -C"/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/"

x booksummaries/
x booksummaries/README
x booksummaries/booksummaries.txt


## Lectura y exploración de datos.

In [9]:
data = []
with open("/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/booksummaries/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in reader:
        data.append(row)

In [10]:
len(data)

16559

In [11]:
title = []
author = []
genre = []
summary = []

for i in range(len(data)):
    title.append(data[i][2])
    author.append(data[i][3])
    if data[i][5] == '':
        genre.append([''])
    else:
        genre.append([j for j in json.loads(data[i][5]).values()])
    summary.append(data[i][6])

df = pd.DataFrame({'Title': title, 'Author': author,
                   'Genre': genre, 'Summary': summary})

print(df.shape)
df.head(5)

(16559, 4)


,Title,Author,Genre,Summary
0,Animal Farm,George Orwell,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca..."
1,A Clockwork Orange,Anthony Burgess,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan..."
2,The Plague,Albert Camus,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...
3,An Enquiry Concerning Human Understanding,David Hume,[],The argument of the Enquiry proceeds by a ser...
4,A Fire Upon the Deep,Vernor Vinge,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16559 entries, 0 to 16558
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Title    16559 non-null  object
 1   Author   16559 non-null  object
 2   Genre    16559 non-null  object
 3   Summary  16559 non-null  object
dtypes: object(4)
memory usage: 517.6+ KB


In [13]:
df[['Title', 'Author']].nunique()

Title     16277
Author     4715
dtype: int64

In [14]:
df['Title'].value_counts().head()

Title
Nemesis     6
Outcast     4
Haunted     4
Inferno     4
The Gift    3
Name: count, dtype: int64

¿Por qué hay mas de un resumen para cada titulo?

In [15]:
df[df['Title'] == 'Nemesis']

,Title,Author,Genre,Summary
375,Nemesis,Isaac Asimov,"[Science Fiction, Speculative fiction, Childre...",The novel is set in an era in which interstel...
3499,Nemesis,Agatha Christie,"[Crime Fiction, Mystery, Children's literature...",Miss Marple receives a post card from the rec...
5157,Nemesis,Scott Ciencin,"[Speculative fiction, Horror]",One of Fred's old friends from graduate schoo...
6159,Nemesis,Jo Nesbø,[Crime Fiction],A bank robbery is committed by a lone robber ...
13696,Nemesis,Philip Roth,[],Nemesis explores the effect of a 1944 polio e...
13842,Nemesis,,[],"The story, set in Latium in AD 77, opens with..."


¿Cuántas categorias tiene la variable 'Genre'?

In [16]:
genre_dict = {}
for i in df['Genre']:
    for j in i:
        if j not in genre_dict:
            genre_dict[j] = 1
        else:
            genre_dict[j] += 1
frec_genre = Counter(genre_dict)

In [17]:
print('Generos distintos: {}\n '.format(len(frec_genre)))

Generos distintos: 228
 


In [18]:
frec_genre.most_common(30)

[('Fiction', 4747),
 ('Speculative fiction', 4314),
 ('', 3718),
 ('Science Fiction', 2870),
 ('Novel', 2463),
 ('Fantasy', 2413),
 ("Children's literature", 2122),
 ('Mystery', 1396),
 ('Young adult literature', 825),
 ('Suspense', 765),
 ('Crime Fiction', 753),
 ('Historical novel', 654),
 ('Thriller', 568),
 ('Horror', 511),
 ('Romance novel', 435),
 ('Historical fiction', 388),
 ('Detective fiction', 341),
 ('Adventure novel', 330),
 ('Non-fiction', 230),
 ('Alternate history', 226),
 ('Spy fiction', 190),
 ('Comedy', 145),
 ('Dystopia', 127),
 ('Autobiography', 124),
 ('Satire', 123),
 ('Gothic fiction', 112),
 ('Comic novel', 104),
 ('Biography', 102),
 ('Novella', 87),
 ('War novel', 87)]

Podemos observar que 3718 resumenes no cuentan con información sobre género del libro.

In [19]:
df['len Summary'] = df['Summary'].apply(lambda x: len(str(x).split()))
df['len Summary'].describe()

count    16559.000000
mean       429.202126
std        500.339692
min          1.000000
25%        120.000000
50%        263.000000
75%        569.000000
max      10334.000000
Name: len Summary, dtype: float64

In [20]:
df.head()

,Title,Author,Genre,Summary,len Summary
0,Animal Farm,George Orwell,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca...",957
1,A Clockwork Orange,Anthony Burgess,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan...",998
2,The Plague,Albert Camus,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...,1119
3,An Enquiry Concerning Human Understanding,David Hume,[],The argument of the Enquiry proceeds by a ser...,2825
4,A Fire Upon the Deep,Vernor Vinge,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...,722


In [21]:
df[df['len Summary'] < 10].sort_values('len Summary')

,Title,Author,Genre,Summary,len Summary
16531,Guardians of Ga'Hoole Book 4: The Siege,Helen Dunmore,"[Speculative fiction, Fantasy, Historical novel]",==Receptio,1
11215,Chucaro: Wild Pony of the Pampa,Francis Kalnay,[Children's literature],==Reference,1
5879,The Caverns of Kalte,Joe Dever,"[Gamebook, Speculative fiction, Fantasy, Child...",==Receptio,1
5693,The Deathlord of Ixia,John Grant,"[Gamebook, Speculative fiction, Children's lit...",==Receptio,1
5972,The Eyes of Darkness,Dean Koontz,"[Speculative fiction, Horror, Fiction, Romance...",==Character,1
...,...,...,...,...,...
13201,Archform: Beauty,"L. E. Modesitt, Jr.",[Science Fiction],Archform: Beauty is set in 24th century Earth.,8
9689,"The Princess Diaries, Volume VII and 3/4: Vale...",Meg Cabot,[Young adult literature],Mia and Michael share Valentine's Day togethe...,9
12201,The Temple of the Ten,H. Bedford-Jones,[Fantasy],The novel adventures in the realms of Prester...,9
12856,The Sword of Aldones,Marion Zimmer Bradley,[Science Fiction],The novel concerns involved intrigue on the p...,9


In [22]:
df = df[df['len Summary'] >= 10].copy().reset_index(drop = True)
df.sort_values('len Summary')

,Title,Author,Genre,Summary,len Summary
11840,The Abyss of Wonders,Perley Poore Sheehan,[Science Fiction],The novel concerns a lost race in the Gobi De...,10
11810,Seeds of Life,Eric Temple Bell,[Science Fiction],The novel concerns the creation of a superman...,10
6395,Bullet Time,David A. McIntee,[Science Fiction],Sarah Jane Smith encounters the Seventh Docto...,10
10853,Stone Tables,Orson Scott Card,"[History, Fiction]",Stone Tables is a novelization of the life of...,10
12356,Yellow Fog,Les Daniels,"[Speculative fiction, Horror]",The novel concerns the vampire Don Sebastian ...,10
...,...,...,...,...,...
14161,March to the Stars,John Ringo,[Science Fiction],The story opens in the restored city of Voita...,6560
12448,Dawkins vs. Gould,Kim Sterelny,[],In the introductory chapter the author points...,7182
14619,Fire World,Chris D'Lacey,[Fantasy],It opens on the planet Co:pern:ica with Couns...,7958
518,"The History of Tom Jones, a Foundling",Henry Fielding,"[Fiction, Novel]",The novel's events occupy eighteen books. Squ...,9055


In [23]:
df.loc[12350, 'Summary']

' Upon reaching the City of Elua, Sidonie and Imriel find that there are many people awaiting them. Some, like the Yeshuites and the Tsingani, are there simply because Imriel was foster-son to Phèdre nó Delaunay and Joscelin Verreuil. Also there are small knots of people, each wearing black armbands around one of their arms, signifying death. They all hold out their fists, thumbs pointed downward. Imriel later learns that these people are the families of his mother, Melisande Shahrizai\'s, victims. In the City, Imriel parts ways with Phèdre and Joscelin, declining their offer to stay at the townhouse in favor of confronting Queen Ysandre. Upon reaching the palace, Imriel comes to find that the Queen has seemed to have cooled off since he had last seen her. His room are ready for his use and he takes the opportunity to have a much-needed bath. Hearing a commotion outside his door, he allows his cousin, Mavros Shahrizai, to be admitted to see him. Mavros comes in and begins berating Imri

## Obtención de los tópicos principales.

## Vectorizacion de textos

In [24]:
def preprocesar(texto):
    # Convertir a minusculas
    texto = (texto).lower()

    # Elimina stopwords
    stop = re.compile(r'\b(' + r'|'.join(stopwords.words('english')) + r')\b\s*')
    texto = stop.sub('', texto)

    # Quitar puntuación y números
    texto = re.sub('[^ña-z]+', ' ', texto)

    # Lemmatizar y quedarnos con palabras que tengan más de tres carácteres
    st = PorterStemmer()
    texto = texto.split()
    texto = ' '.join([st.stem(i) for i in texto if len(i) > 2])

    return(texto)


In [25]:
df['Summary_pp'] = df['Summary'].apply(preprocesar)
df.head()

,Title,Author,Genre,Summary,len Summary,Summary_pp
0,Animal Farm,George Orwell,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca...",957,old major old boar manor farm call anim farm m...
1,A Clockwork Orange,Anthony Burgess,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan...",998,alex teenag live near futur england lead gang ...
2,The Plague,Albert Camus,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...,1119,text plagu divid five part town oran thousand ...
3,An Enquiry Concerning Human Understanding,David Hume,[],The argument of the Enquiry proceeds by a ser...,2825,argument enquiri proce seri increment step sep...
4,A Fire Upon the Deep,Vernor Vinge,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...,722,novel posit space around milki way divid conce...


In [26]:
vectorizer = CountVectorizer(min_df = 10, max_df = 0.10, ngram_range = (1, 2))
BOW = vectorizer.fit_transform(df['Summary_pp'])
BOW.shape

(16496, 37122)

In [27]:
vocabulario = vectorizer.get_feature_names_out()
len(vocabulario)

37122

In [28]:
vocabulario[200:240]

array(['accident stumbl', 'acclaim', 'accommod', 'accompani',
       'accompani back', 'accompani father', 'accompani journey',
       'accompani two', 'accomplic', 'accomplish', 'accomplish goal',
       'accomplish mission', 'accomplish task', 'accord', 'accord author',
       'accord book', 'accord plan', 'accordingli', 'accost', 'account',
       'account event', 'account life', 'accumul', 'accur', 'accuraci',
       'accus', 'accus murder', 'accus steal', 'accus treason',
       'accustom', 'ace', 'ach', 'achiev', 'achiev goal',
       'achiev success', 'achil', 'acid', 'ackbar', 'acknowledg',
       'acolyt'], dtype=object)

## Entrenamiento del modelo

El número óptimo de topicos depende de las caracteristicas del texto a analizar (el largo de los textos, la cantidad de distintas ideas)

No obstante existen algunas metricas que ayudan a determinar k.

In [29]:
k = 10

In [30]:
lda_model = LatentDirichletAllocation(n_components = k, learning_method = 'online', random_state = 42, max_iter = 50)

Se comenta la linea que sigue debido a que se ha guardado el entrenamiento en un archivo pickle. (Descomentar si se quiere volver a reentrenar el modelo con un k diferente)

In [ ]:
## %%time
# lda_model.fit(BOW) # Entrena el modelo y obtienela matriz documento-topico

CPU times: user 9min 51s, sys: 10min, total: 19min 51s
Wall time: 3min 1s


LatentDirichletAllocation(learning_method='online', max_iter=50,
                          random_state=42)

### Guardamos el modelo con pickle

In [ ]:
path = '/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/'
tuple_models = (lda_model, BOW, vectorizer)
pickle.dump(tuple_models, open (path + "tuple_model_books_k10.pkl", 'wb'))

In [31]:
path = '/Users/danielml/Documents/Computational/Curso_Bourbaki/Semana_7/'
lda_model, BOW, vectorizer = pickle.load(open(path + "tuple_model_books_k10.pkl", 'rb'))

## Distribución de temas en cada resumen ($β$)

In [32]:
doc_top = pd.DataFrame(lda_model.transform(BOW))
print(doc_top.shape)
doc_top.head()

(16496, 10)


,0,1,2,3,4,5,6,7,8,9
0,0.032797,0.051167,0.481432,0.175600,0.030791,0.000227,0.000227,0.138991,0.044412,0.044356
1,0.013268,0.000204,0.152536,0.000204,0.000204,0.154217,0.000204,0.103200,0.072890,0.503075
2,0.078117,0.083671,0.240716,0.071468,0.000222,0.088561,0.050103,0.228708,0.158212,0.000222
3,0.000091,0.000091,0.740925,0.027551,0.000091,0.061346,0.047996,0.078882,0.042938,0.000091
4,0.091719,0.021369,0.048427,0.000311,0.000311,0.008979,0.815066,0.013198,0.000311,0.000311


In [33]:
doc_top.sum(axis = 1)

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
16491    1.0
16492    1.0
16493    1.0
16494    1.0
16495    1.0
Length: 16496, dtype: float64

In [34]:
df_lda = pd.merge(df, doc_top, left_index = True, right_index = True)
df_lda.head()

,Title,Author,Genre,Summary,len Summary,Summary_pp,0,1,2,3,4,5,6,7,8,9
0,Animal Farm,George Orwell,"[Roman à clef, Satire, Children's literature, ...","Old Major, the old boar on the Manor Farm, ca...",957,old major old boar manor farm call anim farm m...,0.032797,0.051167,0.481432,0.175600,0.030791,0.000227,0.000227,0.138991,0.044412,0.044356
1,A Clockwork Orange,Anthony Burgess,"[Science Fiction, Novella, Speculative fiction...","Alex, a teenager living in near-future Englan...",998,alex teenag live near futur england lead gang ...,0.013268,0.000204,0.152536,0.000204,0.000204,0.154217,0.000204,0.103200,0.072890,0.503075
2,The Plague,Albert Camus,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...,1119,text plagu divid five part town oran thousand ...,0.078117,0.083671,0.240716,0.071468,0.000222,0.088561,0.050103,0.228708,0.158212,0.000222
3,An Enquiry Concerning Human Understanding,David Hume,[],The argument of the Enquiry proceeds by a ser...,2825,argument enquiri proce seri increment step sep...,0.000091,0.000091,0.740925,0.027551,0.000091,0.061346,0.047996,0.078882,0.042938,0.000091
4,A Fire Upon the Deep,Vernor Vinge,"[Hard science fiction, Science Fiction, Specul...",The novel posits that space around the Milky ...,722,novel posit space around milki way divid conce...,0.091719,0.021369,0.048427,0.000311,0.000311,0.008979,0.815066,0.013198,0.000311,0.000311


## Distribución de palabras en cada tema ($\theta$)

In [35]:
lda_model.perplexity

<bound method LatentDirichletAllocation.perplexity of LatentDirichletAllocation(learning_method='online', max_iter=50,
                          random_state=42)>

In [36]:
θs = pd.DataFrame(lda_model.exp_dirichlet_component_, columns = vocabulario)
print(θs.shape)
θs

(10, 37122)


,aaron,aback,abandon,abandon child,abandon church,abandon citi,abandon famili,abandon farm,abandon group,abandon home,...,zoe,zoey,zola,zombi,zombi like,zone,zoo,zuckerman,zulu,zurich
0,1.355498e-10,1.356356e-10,8.111187e-04,1.356291e-10,1.358826e-10,1.354870e-10,1.354017e-10,1.359244e-10,4.357188e-05,1.357232e-10,...,1.355550e-10,1.352000e-10,1.352381e-10,1.354298e-10,1.355846e-10,1.164478e-04,1.984918e-05,1.352016e-10,1.851751e-05,1.357418e-10
1,3.070025e-04,8.138796e-11,9.502357e-04,2.065533e-05,8.112220e-11,8.140782e-11,8.121868e-11,3.451142e-05,8.109617e-11,8.112871e-11,...,1.542362e-04,8.092416e-11,8.098876e-11,8.100732e-11,8.099291e-11,8.106464e-11,8.115582e-11,8.104724e-11,8.099705e-11,8.132636e-11
2,7.058952e-11,6.040138e-05,3.347842e-04,7.044255e-11,7.043288e-11,7.076264e-11,7.072868e-11,7.065276e-11,7.058500e-11,7.078434e-11,...,7.046038e-11,7.041263e-11,5.481145e-05,7.050952e-11,7.049319e-11,2.166476e-05,7.079634e-05,7.059045e-11,7.047087e-11,7.053272e-11
3,1.348652e-04,7.850058e-06,4.341682e-04,7.280481e-11,7.360871e-11,3.655307e-05,7.277956e-11,7.310606e-11,7.282042e-11,2.253673e-05,...,7.287850e-11,7.273261e-11,7.271236e-11,1.255581e-05,7.291286e-11,7.281637e-11,7.306110e-11,7.271480e-11,7.303678e-11,7.274827e-11
4,4.843314e-10,4.853600e-10,4.994452e-10,4.834587e-10,4.843427e-10,4.836513e-10,4.838551e-10,4.841726e-10,4.841051e-10,4.838029e-10,...,4.846546e-10,3.297030e-03,4.833870e-10,4.840480e-10,4.845398e-10,4.837177e-10,4.846400e-10,4.834086e-10,4.853142e-10,4.853453e-10
5,1.737892e-10,1.741370e-10,5.768488e-04,1.737303e-10,1.738560e-10,1.734864e-10,1.501340e-04,1.739269e-10,1.735084e-10,1.743208e-10,...,1.739074e-10,1.733114e-10,1.733429e-10,5.395855e-04,5.698358e-05,1.735579e-10,5.434565e-05,1.734932e-10,1.739134e-10,1.737664e-10
6,1.323559e-10,1.319668e-10,8.714662e-04,1.317152e-10,1.319137e-10,1.326960e-10,1.320224e-10,1.335075e-10,1.320693e-10,1.318225e-10,...,1.320955e-10,1.317033e-10,1.317111e-10,7.455190e-04,1.331664e-10,9.800475e-04,1.534890e-04,1.318458e-10,1.317998e-10,1.318118e-10
7,8.986888e-11,9.011798e-11,4.028629e-04,7.562188e-06,8.992275e-11,9.009651e-11,8.995602e-11,8.990773e-11,9.002228e-11,1.001535e-05,...,8.984491e-11,8.976231e-11,2.848470e-05,8.988080e-11,8.978677e-11,8.998324e-11,8.998222e-11,8.984001e-11,3.137060e-06,9.032393e-11
8,9.321567e-05,1.098619e-10,1.480112e-04,1.097574e-10,1.097790e-10,1.101748e-10,1.102483e-10,1.109483e-10,1.100346e-10,1.098552e-10,...,1.099043e-10,1.096763e-10,1.098575e-10,1.098808e-10,1.098595e-10,1.101040e-10,1.099159e-10,1.099393e-10,7.761681e-05,2.676966e-05
9,8.032495e-11,8.045586e-11,2.617053e-04,8.016150e-11,2.844392e-05,8.010711e-11,8.029343e-11,8.025329e-11,8.019729e-11,8.083726e-11,...,1.906665e-04,8.006726e-11,8.009350e-11,8.026008e-11,8.033223e-11,8.027071e-11,2.082789e-04,9.227851e-05,8.026694e-11,4.446689e-05


In [37]:
for top in range(k):
    print('\nPalabras más frecuentes del tópico {}'.format(top))
    tokMasFrec = θs.T.loc[:, top].sort_values(ascending = False).head(10).index
    for tok in tokMasFrec:
        print(tok)


Palabras más frecuentes del tópico 0
ship
island
captain
command
soldier
crew
sea
armi
prison
boat

Palabras más frecuentes del tópico 1
mr
john
david
mari
babi
peter
aunt
paul
spend
care

Palabras más frecuentes del tópico 2
chapter
narrat
author
natur
reader
discuss
god
self
women
view

Palabras más frecuentes del tópico 3
king
magic
dragon
lord
god
dark
armi
queen
forest
anim

Palabras más frecuentes del tópico 4
harri
simon
alic
luke
georg
ghote
freddi
val
jedi
poirot

Palabras más frecuentes del tópico 5
jack
tom
vampir
thoma
jim
room
laura
rachel
anita
sarah

Palabras más frecuentes del tópico 6
earth
planet
space
ship
doctor
alien
system
race
star
univers

Palabras más frecuentes del tópico 7
marriag
ladi
sir
letter
london
arthur
england
king
henri
richard

Palabras más frecuentes del tópico 8
govern
unit
american
polit
nation
unit state
presid
countri
militari
soviet

Palabras más frecuentes del tópico 9
polic
car
job
case
york
new york
miss
alex
room
apart


## Visualización del modelo 

In [38]:
pyLDAvis.enable_notebook()

prepared_data = pyLDAvis.lda_model.prepare(lda_model, BOW, vectorizer)
display(prepared_data)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.145123 -0.158993       1        1  14.897008
3     -0.021169  0.018473       2        1  14.359971
9     -0.095660 -0.037677       3        1  12.908271
1     -0.171507 -0.152003       4        1  12.872834
7     -0.038843 -0.104484       5        1  11.764184
8      0.241327 -0.018349       6        1   9.470636
6      0.171802  0.039474       7        1   7.949240
0      0.059205  0.118273       8        1   7.719989
5     -0.209317 -0.033367       9        1   5.945572
4     -0.080961  0.328652      10        1   2.112296, topic_info=            Term         Freq        Total Category  logprob  loglift
29476       ship  5080.000000  5080.000000  Default  30.0000  30.0000
9290       earth  3482.000000  3482.000000  Default  29.0000  29.0000
14570      harri  1635.000000  1635.000000  Default  28.0000  28.0000
17465       king  4428.000000  4428.000000  Default  27.0000  27.0000
25146     planet  2577.000000  2577.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
633         alan   245.101035   339.447121  Topic10  -5.5068   3.5317
16444  inspector   223.038974   421.857531  Topic10  -5.6011   3.2201
5032         cat   271.111809  1343.205345  Topic10  -5.4059   2.2571
20473     master   220.824785  1774.564496  Topic10  -5.6111   1.7735
27246     republ   196.560568   482.568757  Topic10  -5.7275   2.9592

[705 rows x 6 columns], token_table=       Topic      Freq     Term
term                           
112        7  0.178386   aboard
112        8  0.791860   aboard
112       10  0.029006   aboard
235        8  0.993906    achil
337        4  0.993682      ada
...      ...       ...      ...
36991      4  0.021824     york
36991      5  0.059573     york
36991      6  0.109708     york
37088     10  0.995828  yuuzhan
37113     10  0.997652     zoey

[1816 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 4, 10, 2, 8, 9, 7, 1, 6, 5])

## Sistema de recomendación usando similitud coseno



<center><img src="https://substackcdn.com/image/fetch/f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2Fc8f875a9-67a9-40fd-8689-47edbd31eb01_1444x1027.jpeg" width="400" height="300">

$$
\text{similitud coseno}(A, B) = \frac{\sum_{i=1}^{n} A_i B_i}{\sqrt{\sum_{i=1}^{n} A_i^2} \cdot \sqrt{\sum_{i=1}^{n} B_i^2}}
$$

In [39]:
def similitud_coseno(a_vector, b_vector):
    '''Calcula la similitud coseno entre los vectores a y b'''
    numerador = np.dot(a_vector, b_vector)

    a_norm = np.sqrt(np.sum(a_vector**2))
    b_norm = np.sqrt(np.sum(b_vector**2))

    denominador = a_norm * b_norm

    similitud_coseno = numerador / denominador

    return similitud_coseno

In [40]:
def documentos_similares(titulo):
    inx = df[df['Title'] == titulo].index[0]
    q_k = doc_top.loc[inx].values
    n = doc_top.shape[0]
    similaridad = {}
    relevantes = {}

    # Calcular similitud coseno
    for doc_inx in range(n):
        if doc_inx == inx:
            continue
        similaridad[doc_inx] = similitud_coseno(q_k, doc_top.loc[doc_inx].values)
    rank = {k:v for k, v in sorted(similaridad.items(), key = lambda x: x[1],
                                   reverse = True)}
    top5 = pd.DataFrame.from_dict(rank, orient = 'index', columns = ['sim_cos']).head()
    recomendaciones = pd.merge(df.iloc[:, 0:3], top5, how = 'right', right_index = True, left_index = True)
    recomendaciones.index = np.arange(1,6)
    return recomendaciones

In [41]:
documentos_similares('Dune')

,Title,Author,Genre,sim_cos
1,The Mandalorian Armor,K. W. Jeter,"[Science Fiction, Speculative fiction]",0.977255
2,Balance Point,Kathy Tyers,"[Science Fiction, Speculative fiction, Fantasy]",0.957015
3,The Fight for Truth,Judy Blundell,"[Science Fiction, Speculative fiction, Childre...",0.954514
4,The Clone Wars,Karen Traviss,[Science Fiction],0.950693
5,Invincible,Troy Denning,"[Science Fiction, Speculative fiction]",0.948524


In [42]:
documentos_similares('The Time Machine')

,Title,Author,Genre,sim_cos
1,The Flames: A Fantasy,Olaf Stapledon,"[Science Fiction, Novel]",0.993844
2,The Great Romance,,"[Science Fiction, Speculative fiction, Utopian...",0.987241
3,Bones of the Earth,Michael Swanwick,"[Science Fiction, Speculative fiction]",0.985852
4,Isaac Asimov's Robot City: Prodigy,Isaac Asimov,[Science Fiction],0.985713
5,Die Vecna Die!,Steve Miller,[Role-playing game],0.985452


In [43]:
documentos_similares('Harry Potter and the Chamber of Secrets')

,Title,Author,Genre,sim_cos
1,Harry Potter and the Philosopher's Stone,J. K. Rowling,"[Children's literature, Fantasy, Speculative f...",0.992247
2,Harry Potter and the Goblet of Fire,J. K. Rowling,[Speculative fiction],0.986299
3,Harry Potter and the Prisoner of Azkaban,J. K. Rowling,"[Fantasy, Speculative fiction, Young adult lit...",0.982123
4,Harry Potter and the Order of the Phoenix,J. K. Rowling,"[Fantasy, Young adult literature, Fiction]",0.981412
5,Harry Potter and the Half-Blood Prince,J. K. Rowling,"[Fantasy, Young adult literature, Fiction]",0.979495
